<a href="https://colab.research.google.com/github/JayabharathiRavi/pyspark/blob/main/DE_SOLUTION_Jayabharathi_Ravi_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DATA ANALYSIS


In [ ]:
!apt-get update -y

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,545 kB]
Hit:13 http://pp

Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run. Therefore, our first task is to
download Java.

In [ ]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Next, we will download and unzip Apache Spark with Hadoop 2.7 to install it.

In [ ]:
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

Setup Environment variables for Java and Spark

In [ ]:
!pip install -q findspark

In [ ]:
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


Then we need to install and import the 'findspark' library that will locate Spark on the system and import it as a regular library.

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

Now, import SparkSession from pyspark.sql and create a SparkSession, which will be the entry point to Spark.

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = (SparkSession.builder.appName("Chess_Analysis").getOrCreate())

In [ ]:
from pyspark.sql.functions import sum, col, desc

In [ ]:
import pandas as pd

## DATA ANALYSIS

### Importing the datasets.

In [ ]:
df_eco=spark.read.option("header",True).csv("/content/eco_codes.csv")
df_eco.printSchema()
df_eco.show(df_eco.count(),truncate=False)

root
 |-- eco: string (nullable = true)
 |-- eco_name: string (nullable = true)
 |-- eco_example: string (nullable = true)
 |-- eco_type: string (nullable = true)
 |-- eco_group: string (nullable = true)

+---+------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+--------+----------------------+
|eco|eco_name                                                          |eco_example                                                                                                                |eco_type|eco_group             |
+---+------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+--------+----------------------+
|A00|Uncommon Opening                                                  |1 g4, a3, h3, etc.                  

In [ ]:
df_history_game=spark.read.option("header",True).csv("/content/chess_wc_history_game_info.csv")
df_history_game.printSchema()
df_history_game.show(df_history_game.count(),truncate=False)

root
 |-- game_id: string (nullable = true)
 |-- game_order: string (nullable = true)
 |-- event: string (nullable = true)
 |-- site: string (nullable = true)
 |-- date_played: string (nullable = true)
 |-- round: string (nullable = true)
 |-- white: string (nullable = true)
 |-- black: string (nullable = true)
 |-- result: string (nullable = true)
 |-- white_elo: string (nullable = true)
 |-- black_elo: string (nullable = true)
 |-- white_title: string (nullable = true)
 |-- black_title: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- winner_elo: string (nullable = true)
 |-- loser: string (nullable = true)
 |-- loser_elo: string (nullable = true)
 |-- winner_loser_elo_diff: string (nullable = true)
 |-- eco: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- tournament_name: string (nullable = true)

+------------------------------------+----------+---------------------------+----------------+-----------+-----+---------------------------+---

In [ ]:
df_history_moves=spark.read.option("header",True).csv("/content/chess_wc_history_moves.csv")

In [ ]:
df_history_moves.printSchema()
df_history_moves.show()

root
 |-- game_id: string (nullable = true)
 |-- move_no: string (nullable = true)
 |-- move_no_pair: string (nullable = true)
 |-- player: string (nullable = true)
 |-- notation: string (nullable = true)
 |-- move: string (nullable = true)
 |-- from_square: string (nullable = true)
 |-- to_square: string (nullable = true)
 |-- piece: string (nullable = true)
 |-- color: string (nullable = true)
 |-- fen: string (nullable = true)
 |-- is_check: string (nullable = true)
 |-- is_check_mate: string (nullable = true)
 |-- is_fifty_moves: string (nullable = true)
 |-- is_fivefold_repetition: string (nullable = true)
 |-- is_game_over: string (nullable = true)
 |-- is_insufficient_material: string (nullable = true)
 |-- white_count: string (nullable = true)
 |-- black_count: string (nullable = true)
 |-- white_pawn_count: string (nullable = true)
 |-- black_pawn_count: string (nullable = true)
 |-- white_queen_count: string (nullable = true)
 |-- black_queen_count: string (nullable = true)
 

Naming the datasets

In [ ]:
df_eco.createOrReplaceTempView('eco')

In [ ]:
df_history_game.createOrReplaceTempView("game")

In [ ]:
df_history_moves.createOrReplaceTempView("move")

1. List of Winners of Each World champions Trophy Hint: Total Result of all rounds of Tournament for that player is considered as that player's
Score/Result.

In [ ]:
df_history_game.show()

In [ ]:
x=df_history_game.filter(df_history_game['winner'] != "draw").groupBy("tournament_name").agg({"winner":"max"})

## **RESULT.1**

In [ ]:
x.show()

+---------------+--------------------+
|tournament_name|         max(winner)|
+---------------+--------------------+
| WorldChamp2004|              Leko,P|
| WorldChamp1894|   Steinitz, William|
| WorldChamp2013|     Carlsen, Magnus|
|  FideChamp2000|       Yermolinsky,A|
| WorldChamp2007|           Svidler,P|
|  FideChamp1993|       Timman, Jan H|
|WorldChamp1910b|     Lasker, Emanuel|
| WorldChamp1921|Capablanca, Jose ...|
| WorldChamp1958|    Smyslov, Vassily|
| WorldChamp1981|  Kortschnoj, Viktor|
| WorldChamp1961|         Tal, Mihail|
| WorldChamp1978|  Kortschnoj, Viktor|
| WorldChamp1960|         Tal, Mihail|
| WorldChamp1948|    Smyslov, Vassily|
| WorldChamp1929|    Bogoljubow, Efim|
| WorldChamp1934|    Bogoljubow, Efim|
| WorldChamp1986|      Kasparov, Gary|
|   PCAChamp1995|      Kasparov, Gary|
| WorldChamp1886|Zukertort, Johann...|
| WorldChamp1907|     Lasker, Emanuel|
+---------------+--------------------+
only showing top 20 rows



In [ ]:
df_1=x.toPandas()
df_1

,tournament_name,max(winner)
0,WorldChamp2004,"Leko,P"
1,WorldChamp1894,"Steinitz, William"
2,WorldChamp2013,"Carlsen, Magnus"
3,FideChamp2000,"Yermolinsky,A"
4,WorldChamp2007,"Svidler,P"
5,FideChamp1993,"Timman, Jan H"
6,WorldChamp1910b,"Lasker, Emanuel"
7,WorldChamp1921,"Capablanca, Jose Raul"
8,WorldChamp1958,"Smyslov, Vassily"
9,WorldChamp1981,"Kortschnoj, Viktor"


2. List of Players with number of times they have won Tournament in descending order(Max to min).
Result attributes: player_name, number_of_wins

In [ ]:
split_col=pyspark.sql.functions.split(df_history_game['winner'],',')
df_new=df_history_game.withColumn('first_name', split_col.getItem(0))\
                      .withColumn('last_name', split_col.getItem(1))

df_new.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|first_name|last_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     28

In [ ]:
split_col=pyspark.sql.functions.split(x['max(winner)'],',')
df_new1=x.withColumn('first_name', split_col.getItem(0))\
                      .withColumn('last_name', split_col.getItem(1))

df_new1.show()

+---------------+--------------------+-----------+-----------------+
|tournament_name|         max(winner)| first_name|        last_name|
+---------------+--------------------+-----------+-----------------+
| WorldChamp2004|              Leko,P|       Leko|                P|
| WorldChamp1894|   Steinitz, William|   Steinitz|          William|
| WorldChamp2013|     Carlsen, Magnus|    Carlsen|           Magnus|
|  FideChamp2000|       Yermolinsky,A|Yermolinsky|                A|
| WorldChamp2007|           Svidler,P|    Svidler|                P|
|  FideChamp1993|       Timman, Jan H|     Timman|            Jan H|
|WorldChamp1910b|     Lasker, Emanuel|     Lasker|          Emanuel|
| WorldChamp1921|Capablanca, Jose ...| Capablanca|        Jose Raul|
| WorldChamp1958|    Smyslov, Vassily|    Smyslov|          Vassily|
| WorldChamp1981|  Kortschnoj, Viktor| Kortschnoj|           Viktor|
| WorldChamp1961|         Tal, Mihail|        Tal|           Mihail|
| WorldChamp1978|  Kortschnoj, Vik

In [ ]:
df_new1.createOrReplaceTempView('wins')

In [ ]:
xy=spark.sql("select first_name as player_name, count(first_name) as number_of_wins from wins group by first_name")

In [ ]:
xy.createOrReplaceTempView('winners')
y=spark.sql('select * from winners ORDER BY number_of_wins desc')

## **RESULT.2**

In [ ]:
y.show()

+-----------+--------------+
|player_name|number_of_wins|
+-----------+--------------+
|   Kasparov|             6|
|   Steinitz|             5|
|    Smyslov|             4|
| Zvjaginsev|             4|
|    Topalov|             3|
|     Lasker|             3|
|    Carlsen|             3|
|    Spassky|             3|
| Kortschnoj|             2|
| Bogoljubow|             2|
|    Kramnik|             2|
|       Euwe|             2|
|        Tal|             2|
| Capablanca|             2|
|  Bronstein|             1|
|  Petrosian|             1|
|    Svidler|             1|
|     Karpov|             1|
|Yermolinsky|             1|
|     Timman|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
df_2=y.toPandas()
df_2

,player_name,number_of_wins
0,Kasparov,6
1,Steinitz,5
2,Smyslov,4
3,Zvjaginsev,4
4,Lasker,3
5,Topalov,3
6,Spassky,3
7,Carlsen,3
8,Kortschnoj,2
9,Kramnik,2


3. Most and Least Popular eco move in world championship history.
Result attributes: eco, eco_name, number_of_occurences
Final result will have only two rows

In [ ]:
df_eco.show()

In [ ]:
split_col=pyspark.sql.functions.split(df_eco['eco_name'],',')
df_new10=df_eco.withColumn('f_eco', split_col.getItem(0))\
                      .withColumn('l_eco', split_col.getItem(1))\
                      .withColumn('t_eco', split_col.getItem(2))


In [ ]:
df_new10.createOrReplaceTempView('new_10')

In [ ]:
a1=spark.sql("select f_eco as eco_name,count(*) as number_of_occurence from new_10 group by f_eco")

In [ ]:
max=a1.sort(col('number_of_occurence').desc())
min=a1.sort(col('number_of_occurence').asc())

In [ ]:
df1=max.toPandas()

In [ ]:
df2=min.toPandas()

In [ ]:
max.show(1)
min.show(1)

+--------+-------------------+
|eco_name|number_of_occurence|
+--------+-------------------+
|Sicilian|                 80|
+--------+-------------------+
only showing top 1 row

+--------------------+-------------------+
|            eco_name|number_of_occurence|
+--------------------+-------------------+
|King's Indian Def...|                  1|
+--------------------+-------------------+
only showing top 1 row



In [ ]:
df_3=df1.iloc[[0],[0,1]].append(df2.iloc[[0],[0,1]],ignore_index=True)
df_3

,eco_name,number_of_occurence
0,Sicilian,80
1,Three Knights,1


4. Find the eco move with most winnings.
Ps. Use this opening move in your next chess game🙂
Result attributes: eco, eco_name

In [ ]:
df_new.createOrReplaceTempView('new')
b=spark.sql("select game_id,first_name from new where first_name not like 'draw'")

In [ ]:
split_col=pyspark.sql.functions.split(df_history_moves['player'],',')
df_new11=df_history_moves.withColumn('fname', split_col.getItem(0))\
                      .withColumn('lname', split_col.getItem(1))

In [ ]:
p=b.join(df_history_moves,b.game_id==df_history_moves.game_id,"inner")
p.show()

In [ ]:
p.createOrReplaceTempView('moveseq')

In [ ]:
a1=spark.sql("select move_sequence,count(*) as num from moveseq group by move_sequence")

In [ ]:
a1.show()

In [ ]:
max=a1.sort(a1.num.desc())
max.show()

In [ ]:
x=df_eco.filter((df_eco.eco_example.like("%e4%")))

In [ ]:
x.show()

In [ ]:
x.createOrReplaceTempView('temp')
x1=spark.sql("select eco_example,count(*) as num from temp group by eco_example")

In [ ]:
max=x1.sort(x1.num.desc())
max.show(max.count(),truncate=False)

In [ ]:
y=df_eco.filter((df_eco.eco_example.like('1 d4 Nf6 2 c4 c5 3 d5 e6 4 Nc3 exd5 5 cxd5 d6 6 e4 g6 7 Nf3 Bg7 8 Be2 O-O 9')))

In [ ]:
y.createOrReplaceTempView('temp1')
y1=spark.sql("select eco,eco_name from temp1 limit 1")

In [ ]:
y1.show()

+---+--------------------+
|eco|            eco_name|
+---+--------------------+
|A73|Benoni, Classical...|
+---+--------------------+



In [ ]:
df_4=y1.toPandas()
df_4

,eco,eco_name
0,A73,"Benoni, Classical, 9.O-O"


5. Longest and shortest game ever played in a world championship in terms of move.
Chess Funda: "move" is completed once both White and Black have played one turn. 

e.g If a game lasts 10 moves, both White and Black have
played 10 moves)

Result attributes: game_id, event, tournament_name, number_of_moves

Final result will have only two rows

In [ ]:
p=spark.sql("select game.game_id as game_id,game.event,game.tournament_name,int(move.move_no_pair) as number_of_move from game INNER JOIN move ON game.game_id=move.game_id;")

In [ ]:
p.createOrReplaceTempView('moving')

In [ ]:
a1=spark.sql("select game_id,max(number_of_move) as number_of_move from moving group by game_id ")

In [ ]:
a2=p.groupby(['game_id','event','tournament_name']).max('number_of_move').alias('number_of_move')

In [ ]:
a2.createOrReplaceTempView('a2')

In [ ]:
max=a1.sort(a1.number_of_move.desc())

In [ ]:
min=a1.sort(a1.number_of_move.asc())

In [ ]:
df1=max.toPandas()
df2=min.toPandas()

In [ ]:
df_55=df1.iloc[[0],[0,1]].append(df2.iloc[[0],[0,1]],ignore_index=True)
df_55

,game_id,number_of_move
0,58e83255-93bb-4d58-8c14-78a991419338,136
1,8ce46a05-02f2-4212-a33d-ceaa7907e00c,8


In [ ]:
df_5=spark.createDataFrame(df_55)
df_5.show()

+--------------------+--------------+
|             game_id|number_of_move|
+--------------------+--------------+
|58e83255-93bb-4d5...|           136|
|8ce46a05-02f2-421...|             8|
+--------------------+--------------+



In [ ]:
df_5.createOrReplaceTempView('df_5')

In [ ]:
p12=spark.sql("select df_5.game_id as game_id,a2.event,a2.tournament_name, df_5.number_of_move from df_5 INNER JOIN a2 ON df_5.game_id=a2.game_id;")

## **RESULT.5**

In [ ]:
p12.show()

+--------------------+--------+---------------+--------------+
|             game_id|   event|tournament_name|number_of_move|
+--------------------+--------+---------------+--------------+
|58e83255-93bb-4d5...|WCh 2021| WorldChamp2021|           136|
|8ce46a05-02f2-421...|WCh-FIDE|  FideChamp2005|             8|
+--------------------+--------+---------------+--------------+



In [ ]:
df_5=p12.toPandas()
df_5

,game_id,event,tournament_name,number_of_move
0,58e83255-93bb-4d58-8c14-78a991419338,WCh 2021,WorldChamp2021,136
1,8ce46a05-02f2-4212-a33d-ceaa7907e00c,WCh-FIDE,FideChamp2005,8


6. Shortest and Longest Draw game ever Played.

Result attributes: game_id, event, tournament_name, number_of_moves

Final result will have only two rows

In [ ]:
df_new.createOrReplaceTempView('game')
df_history_moves.createOrReplaceTempView('move')

In [ ]:
p=spark.sql("select game.game_id as game_id,game.event,game.tournament_name,game.first_name,int(move.move_no_pair) as number_of_move from game INNER JOIN move ON game.game_id=move.game_id;")

In [ ]:
p.createOrReplaceTempView('moving')

In [ ]:
a4=spark.sql("select game_id,event,tournament_name,number_of_move from moving where first_name like 'draw' ")

In [ ]:
a4.createOrReplaceTempView('move_draw')

In [ ]:
a1=spark.sql("select game_id,max(number_of_move) as number_of_move from move_draw group by game_id ")

In [ ]:
a1.show()

In [ ]:
a2=a4.groupby(['game_id','event','tournament_name']).max('number_of_move').alias('number_of_move')

In [ ]:
a2.createOrReplaceTempView('a2')

In [ ]:
max=a1.sort(a1.number_of_move.desc())

In [ ]:
min=a1.sort(a1.number_of_move.asc())

In [ ]:
df1=max.toPandas()
df2=min.toPandas()

In [ ]:
df_55=df1.iloc[[0],[0,1]].append(df2.iloc[[0],[0,1]],ignore_index=True)
df_55

,game_id,number_of_move
0,99d39a93-427e-4d1a-bd36-ff23f98829e5,115
1,df7f5f8b-2e0a-4906-8bbc-6d1837594b5b,30


In [ ]:
df_5=spark.createDataFrame(df_55)
df_5.show()

+--------------------+--------------+
|             game_id|number_of_move|
+--------------------+--------------+
|99d39a93-427e-4d1...|           115|
|df7f5f8b-2e0a-490...|            30|
+--------------------+--------------+



In [ ]:
df_5.createOrReplaceTempView('df_5')

In [ ]:
p12=spark.sql("select df_5.game_id as game_id,a2.event,a2.tournament_name, df_5.number_of_move from df_5 INNER JOIN a2 ON df_5.game_id=a2.game_id;")

## **RESULT.6**

In [ ]:
p12.show()

+--------------------+--------+---------------+--------------+
|             game_id|   event|tournament_name|number_of_move|
+--------------------+--------+---------------+--------------+
|99d39a93-427e-4d1...|WCh 2018| WorldChamp2018|           115|
|df7f5f8b-2e0a-490...|WCh 2016| WorldChamp2016|            30|
+--------------------+--------+---------------+--------------+



In [ ]:
df_6=p12.toPandas()
df_6

,game_id,event,tournament_name,number_of_move
0,99d39a93-427e-4d1a-bd36-ff23f98829e5,WCh 2018,WorldChamp2018,115
1,df7f5f8b-2e0a-4906-8bbc-6d1837594b5b,WCh 2016,WorldChamp2016,30


7. Most and Least rated Player.

Result attributes: player_name, elo

Chess Funda: elo is the rating of the player in chess tournament.

Final result will have only two rows

In [ ]:
split_col=pyspark.sql.functions.split(df_new['white'],',')
df_new2=df_new.withColumn('f_name', split_col.getItem(0))\
                      .withColumn('l_name', split_col.getItem(1))


In [ ]:
split_col=pyspark.sql.functions.split(df_new['black'],',')
df_new3=df_new2.withColumn('bf_name', split_col.getItem(0))\
                      .withColumn('bl_name', split_col.getItem(1))

df_new3.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+--------+--------------+--------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|first_name|last_name|        f_name|  l_name|       bf_name| bl_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+--------+-----------

In [ ]:
df_new3.createOrReplaceTempView('elo')

In [ ]:
zw=spark.sql("select f_name as player_name,avg(white_elo) as elo from elo group by f_name ")
zb=spark.sql("select bf_name as player_name,avg(black_elo) as elo from elo group by bf_name ")

zw.show()
zb.show()

+--------------+------------------+
|   player_name|               elo|
+--------------+------------------+
|Becerra Rivero|            2546.0|
|     Giorgadze|            2625.0|
|         Milov| 2619.076923076923|
|        Elarbi|            2257.0|
|        Babula|            2567.0|
|         Kritz|            2534.0|
|       Fominyh|            2535.0|
|        Asabri|            2277.0|
|    Morozevich|2725.5416666666665|
|        Kobese|            2374.0|
|  Peng Xiaomin|            2610.0|
|      Karjakin|2750.6666666666665|
|     Neelotpal|            2457.0|
|       Psakhis| 2587.769230769231|
|         Safin|            2538.0|
|          Lima|2530.6666666666665|
|        Lerner|            2560.0|
|   Gu Xiaobing|            2299.0|
|         Gulko|2623.4285714285716|
|     Arakhamia|            2437.0|
+--------------+------------------+
only showing top 20 rows

+--------------+------------------+
|   player_name|               elo|
+--------------+------------------+
|B

Almost two table shows equal elo for the person.So, we can take any one of the table for future process.

In [ ]:
max = zw.orderBy(col('elo').desc())
min = zw.orderBy(col('elo').asc())

In [ ]:
max.show(1)
min.show(1)

+-----------+----------+
|player_name|       elo|
+-----------+----------+
|    Carlsen|2835.65625|
+-----------+----------+
only showing top 1 row

+-----------+----+
|player_name| elo|
+-----------+----+
|   Zeliakov|null|
+-----------+----+
only showing top 1 row



In [ ]:
df1=max.toPandas()
df2=min.toPandas()

## **RESULT.7**

In [ ]:
df=df1.iloc[[0],[0,1]].append(df2.iloc[[0],[0,1]],ignore_index=True)
df_7=df.fillna(0)
df_7

,player_name,elo
0,Carlsen,2835.65625
1,Zeliakov,0.00000


In [ ]:
df_2_7=spark.createDataFrame(df_7)
df_2_7.show()

+-----------+----------+
|player_name|       elo|
+-----------+----------+
|    Carlsen|2835.65625|
|   Zeliakov|       0.0|
+-----------+----------+



8. 3rd Last Player with most Loss.

Result attributes: player_name

Final result will have only one row

In [ ]:
split_col=pyspark.sql.functions.split(df_new['loser'],',')
df_newloser=df_new.withColumn('lf_name', split_col.getItem(0))\
                      .withColumn('ll_name', split_col.getItem(1))
df_newloser.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+-------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|first_name|last_name|       lf_name|ll_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+-------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11

In [ ]:
loss=df_newloser.filter(df_newloser['lf_name'] != "draw").groupBy("tournament_name").agg({"lf_name":"max"})

In [ ]:
loss.show()

+---------------+------------+
|tournament_name|max(lf_name)|
+---------------+------------+
| WorldChamp2004|        Leko|
| WorldChamp1894|    Steinitz|
| WorldChamp2013|       Anand|
|  FideChamp2000|        Zaja|
| WorldChamp2007|     Svidler|
|  FideChamp1993|      Timman|
|WorldChamp1910b|    Janowsky|
| WorldChamp1921|      Lasker|
| WorldChamp1958|     Smyslov|
| WorldChamp1981|  Kortschnoj|
| WorldChamp1961|         Tal|
| WorldChamp1978|  Kortschnoj|
| WorldChamp1960|         Tal|
| WorldChamp1948|     Smyslov|
| WorldChamp1929|  Bogoljubow|
| WorldChamp1934|  Bogoljubow|
| WorldChamp1986|    Kasparov|
|   PCAChamp1995|    Kasparov|
| WorldChamp1886|   Zukertort|
| WorldChamp1907|    Marshall|
+---------------+------------+
only showing top 20 rows



In [ ]:
df_newloser.createOrReplaceTempView('lose')

In [ ]:
l=spark.sql("select loser as player_name,count(*)as number from game where loser not like 'draw' group by loser ")

In [ ]:
max_l = l.orderBy(col('number').desc())
max_l.show(3)

+------------------+------+
|       player_name|number|
+------------------+------+
| Steinitz, William|    43|
|Botvinnik, Mikhail|    41|
|   Karpov, Anatoly|    36|
+------------------+------+
only showing top 3 rows



In [ ]:
dfl=max_l.toPandas()
dfl.head(5)

,player_name,number
0,"Steinitz, William",43
1,"Botvinnik, Mikhail",41
2,"Karpov, Anatoly",36
3,"Euwe, Max",31
4,"Alekhine, Alexander",24


In [ ]:
a=dfl.iloc[2]
a

player_name    Karpov, Anatoly
number                      36
Name: 2, dtype: object

## **RESULT.8**

In [ ]:
_3rdlast=pd.DataFrame(data=['Karpov, Anatoly'],columns =['player_name'])
_3rdlast

,player_name
0,"Karpov, Anatoly"


In [ ]:
df_8=spark.createDataFrame(_3rdlast)
df_8.show()

+---------------+
|    player_name|
+---------------+
|Karpov, Anatoly|
+---------------+



9. How many times players with low rating won matches with their total win Count.

Result attributes: player_name, win_count

In [ ]:
df_new.createOrReplaceTempView('wincount')

In [ ]:
w=spark.sql("select int(winner_elo), int(loser_elo),first_name from wincount where first_name not like 'draw' ")

In [ ]:
w.createOrReplaceTempView('winning')

In [ ]:
w1=spark.sql("select int(loser_elo)as win_count , first_name as person_name from winning where int(loser_elo)<int(winner_elo)")

In [ ]:
a3=w1.groupby('person_name').sum('win_count')

## **RESULT.9**

In [ ]:
a3.show()

+-------------+--------------+
|  person_name|sum(win_count)|
+-------------+--------------+
|    Giorgadze|          2545|
|        Milov|         10281|
|       Babula|          2471|
|   Morozevich|         28813|
| Peng Xiaomin|          7634|
|      Psakhis|          4981|
|        Gulko|          7654|
|   Hjartarson|          2510|
|     Tkachiev|         17702|
|      Lautier|         20354|
|      Nijboer|          5010|
|Granda Zuniga|          5090|
|       Serper|          7543|
|     Kasparov|         70190|
|       Jobava|          5184|
|     Benjamin|          7396|
|      Antonio|          4700|
|     Nenashev|          2400|
|       Xu Jun|          5126|
|    Andersson|         12730|
+-------------+--------------+
only showing top 20 rows



In [ ]:
df_9=a3.toPandas()
df_9

,person_name,sum(win_count)
0,Giorgadze,2545
1,Milov,10281
2,Babula,2471
3,Morozevich,28813
4,Peng Xiaomin,7634
...,...,...
133,Tiviakov,10214
134,Vladimirov,7591
135,Anastasian,2457
136,Dominguez,7762


10. Move Sequence for Each Player in a Match.

Result attributes: game_id, player_name, move_sequence, move_count

In [ ]:
split_col=pyspark.sql.functions.split(df_history_moves['player'],',')
df_new5=df_history_moves.withColumn('pf_name', split_col.getItem(0))\
                      .withColumn('pl_name', split_col.getItem(1))

df_new5.show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
df_new5.createOrReplaceTempView('seq')

In [ ]:
a4=spark.sql("select pf_name as player_name,sum(int(move_no))as move_count from seq group by pf_name")

## **RESUL.10**

In [ ]:
a4.show()

+--------------+----------+
|   player_name|move_count|
+--------------+----------+
|    Morozevich|     66767|
|      Karjakin|     50761|
|  Kasimdzhanov|     20827|
|        Polgar|     22877|
|       Caruana|     39889|
|         Anand|    179842|
|       Kramnik|     74180|
|       Svidler|     32278|
|         Adams|     16988|
|Nepomniachtchi|     37706|
|       Topalov|    109369|
|      Grischuk|     29648|
|          Leko|     49831|
|       Carlsen|    180552|
|       Gelfand|     52695|
|       Aronian|     23489|
+--------------+----------+



In [ ]:
df_10=a4.toPandas()
df_10

,player_name,move_count
0,Morozevich,66767
1,Karjakin,50761
2,Kasimdzhanov,20827
3,Polgar,22877
4,Caruana,39889
5,Anand,179842
6,Kramnik,74180
7,Svidler,32278
8,Adams,16988
9,Nepomniachtchi,37706


11. Total Number of games where losing player has more Captured score than Winning player.

Hint: Captured score is cumulative, i.e., for 3rd capture it will have score for 1, 2, and 3rd.

Result attributes: total_number_of_games Final result will have only one row

In [ ]:
b=spark.sql("select game_id,lf_name from lose where int(loser_elo)>int(winner_elo) ")

In [ ]:
loss1=b.filter(b['lf_name'] != "draw").groupBy("game_id").agg({"lf_name":"max"})
loss1.show()

+--------------------+-------------+
|             game_id| max(lf_name)|
+--------------------+-------------+
|41ae3e1c-9c3c-408...|       Luther|
|8bc3b89a-6603-415...| Vallejo Pons|
|ebc5ddeb-12ba-4c6...|        Adams|
|96b82357-aa9d-44c...|         Leko|
|9d5f2451-21ca-4fd...|    Khalifman|
|aeeb14ca-b591-483...|         Leko|
|c77671af-4a21-4e3...|      Aronian|
|9f944a46-6cc4-40d...|Granda Zuniga|
|a828ea03-b4e9-491...|  Zhang Zhong|
|5135b452-5d4b-4e9...|        Anand|
|c2ad1d49-8a47-428...|        Short|
|71f22630-4b08-460...|  Xu Yuanyuan|
|7ea5cbe6-7f26-42b...|         Leko|
|822023b0-1f99-456...|      Kramnik|
|503deec8-84fe-4b6...|       Karpov|
|2b0266de-b1b4-466...|       Kudrin|
|a0648099-1b4c-40d...|     Vaganian|
|1de806bd-8328-4ab...|    Beliavsky|
|41639ad8-5ba4-4fa...|       Jobava|
|5bc21866-5bd5-4a0...|        Dreev|
+--------------------+-------------+
only showing top 20 rows



In [ ]:
loss1.createOrReplaceTempView('loserhigh')
h=spark.sql("select count(game_id) as total_number_of_games from loserhigh")

## **RESULT.11**

In [ ]:
h.show()

+---------------------+
|total_number_of_games|
+---------------------+
|                  347|
+---------------------+



In [ ]:
df_11=h.toPandas()
df_11

,total_number_of_games
0,347


12. List All Perfect Tournament with Winner Name.

Chess Funda: Perfect Tournament means a player has won all the matches excluding draw matches. e.g Player A has won 5 matches out of 7
Matches in tournament where 2 matches are draw and player B has won 0 matches)

Result attributes: winner_name, tournament_name

In [ ]:
split_col=pyspark.sql.functions.split(df_new['white'],',')
df_new2=df_new.withColumn('f_name', split_col.getItem(0))\
                      .withColumn('l_name', split_col.getItem(1))

split_col=pyspark.sql.functions.split(df_new['black'],',')
df_new3=df_new2.withColumn('bf_name', split_col.getItem(0))\
                      .withColumn('bl_name', split_col.getItem(1))


In [ ]:
df_new3.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+--------+--------------+--------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|first_name|last_name|        f_name|  l_name|       bf_name| bl_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+--------+-----------

In [ ]:
df_new3.createOrReplaceTempView('elo')

In [ ]:
f=spark.sql("select first_name,tournament_name from elo where first_name not like 'draw' ")

In [ ]:
f1=spark.sql("select f_name,bf_name,tournament_name from elo where first_name like 'draw' ")

In [ ]:
f.createOrReplaceTempView('f1')
f1.createOrReplaceTempView('f2')

In [ ]:
df_new3.createOrReplaceTempView('elo')

In [ ]:
f=spark.sql("select first_name,tournament_name from elo where first_name not like 'draw' ")

In [ ]:
f.show()

In [ ]:
f1=spark.sql("select f_name,bf_name,tournament_name from elo where first_name like 'draw' ")

In [ ]:
f1.show()

In [ ]:
f.createOrReplaceTempView('f1')
f1.createOrReplaceTempView('f2')

In [ ]:
f3=spark.sql('select tournament_name from f2 group by tournament_name').show()

+---------------+
|tournament_name|
+---------------+
| WorldChamp2004|
| WorldChamp1894|
| WorldChamp2013|
| WorldChamp2007|
|  FideChamp2000|
|  FideChamp1993|
|WorldChamp1910b|
| WorldChamp1921|
| WorldChamp1958|
| WorldChamp1981|
| WorldChamp1978|
| WorldChamp1961|
| WorldChamp1960|
| WorldChamp1948|
| WorldChamp1929|
| WorldChamp1934|
| WorldChamp1986|
|   PCAChamp1995|
| WorldChamp1886|
| WorldChamp1951|
+---------------+
only showing top 20 rows



In [ ]:
f4=spark.sql('select f_name from f2 group by f_name').show()

+------------+
|      f_name|
+------------+
|   Giorgadze|
|       Milov|
|      Babula|
|  Morozevich|
|Peng Xiaomin|
|      Kobese|
|    Karjakin|
|   Neelotpal|
|       Safin|
|     Psakhis|
|        Lima|
|       Gulko|
|  Hjartarson|
|      Volkov|
|    Tkachiev|
| Miladinovic|
|       Salov|
|     Lautier|
|  Timoshenko|
|     Nijboer|
+------------+
only showing top 20 rows



## **Result.12**

In [ ]:
spark.sql("select f_name as name,bf_name as name ,tournament_name from f2").show()

+--------------+--------------+---------------+
|          name|          name|tournament_name|
+--------------+--------------+---------------+
|Nepomniachtchi|       Carlsen| WorldChamp2021|
|       Carlsen|Nepomniachtchi| WorldChamp2021|
|Nepomniachtchi|       Carlsen| WorldChamp2021|
|       Carlsen|Nepomniachtchi| WorldChamp2021|
|Nepomniachtchi|       Carlsen| WorldChamp2021|
|Nepomniachtchi|       Carlsen| WorldChamp2021|
|       Carlsen|Nepomniachtchi| WorldChamp2021|
|       Caruana|       Carlsen| WorldChamp2018|
|       Carlsen|       Caruana| WorldChamp2018|
|       Caruana|       Carlsen| WorldChamp2018|
|       Carlsen|       Caruana| WorldChamp2018|
|       Caruana|       Carlsen| WorldChamp2018|
|       Carlsen|       Caruana| WorldChamp2018|
|       Carlsen|       Caruana| WorldChamp2018|
|       Caruana|       Carlsen| WorldChamp2018|
|       Carlsen|       Caruana| WorldChamp2018|
|       Caruana|       Carlsen| WorldChamp2018|
|       Carlsen|       Caruana| WorldCha

13. Player with highest winning ratio.

Hint: Winning ratio: (Number of rounds won)/(Number of rounds played)

Result attributes: player_name
Final result will have only one row

In [ ]:
a3.show()

+-------------+--------------+
|  person_name|sum(win_count)|
+-------------+--------------+
|    Giorgadze|          2545|
|        Milov|         10281|
|       Babula|          2471|
|   Morozevich|         28813|
| Peng Xiaomin|          7634|
|      Psakhis|          4981|
|        Gulko|          7654|
|   Hjartarson|          2510|
|     Tkachiev|         17702|
|      Lautier|         20354|
|      Nijboer|          5010|
|Granda Zuniga|          5090|
|       Serper|          7543|
|     Kasparov|         70190|
|       Jobava|          5184|
|     Benjamin|          7396|
|      Antonio|          4700|
|     Nenashev|          2400|
|       Xu Jun|          5126|
|    Andersson|         12730|
+-------------+--------------+
only showing top 20 rows



In [ ]:
df_history_moves.createOrReplaceTempView('move')
a4=spark.sql("select player,count(player) from move group by player")

In [ ]:
a4=spark.sql("select player,count(round) from move group by round")

In [ ]:
df_a3=a3.toPandas()
df_a4=a4.toPandas()

In [ ]:
df_history_game.show()

In [ ]:
df  = pd.concat([df_a3, df_a4], axis=1)
df

,person_name,sum(win_count),player,count(player)
0,Giorgadze,2545,"Nepomniachtchi,I",568.0
1,Milov,10281,"Carlsen,M",568.0
2,Babula,2471,"Adams,Mi",384.0
3,Morozevich,28813,"Leko,P",999.0
4,Peng Xiaomin,7634,"Kasimdzhanov,R",434.0
...,...,...,...,...
133,Tiviakov,10214,NaN,NaN
134,Vladimirov,7591,NaN,NaN
135,Anastasian,2457,NaN,NaN
136,Dominguez,7762,NaN,NaN


In [ ]:
a

14. Player who had given checkmate with Pawn.

Note: Consider all events for this query

Result attributes: player_name
Final result will have only one row

In [ ]:
df_new3.show()

+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+--------+--------------+--------+
|             game_id|game_order|   event|      site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|first_name|last_name|        f_name|  l_name|       bf_name| bl_name|
+--------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+----------+---------+--------------+--------+-----------

In [ ]:
df_new5.createOrReplaceTempView('df_5')

In [ ]:
df_new3.createOrReplaceTempView('df3')

In [ ]:
df_new5.show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
p12=spark.sql("select df_5.game_id as game_id,df3.event,df3.f_name, df_5.is_check_mate,df_5.is_check from df_5 INNER JOIN df3 ON df_5.game_id=df3.game_id;")

In [ ]:
p12.createOrReplaceTempView('paw')

In [ ]:
paw=spark.sql("select f_name as player_name from paw where paw.is_check like '1' ")

In [ ]:
paw.createOrReplaceTempView('p')
pq=spark.sql("select count(player_name) as count,player_name from p group by player_name")

In [ ]:
pq.show()

+-----+--------------+
|count|   player_name|
+-----+--------------+
|   67|    Morozevich|
|   28|      Karjakin|
|   29|  Kasimdzhanov|
|   26|        Polgar|
|   30|       Caruana|
|  170|         Anand|
|   81|       Kramnik|
|   29|       Svidler|
|   20|         Adams|
|   18|Nepomniachtchi|
|  136|       Topalov|
|   25|      Grischuk|
|   61|          Leko|
|  207|       Carlsen|
|   27|       Gelfand|
|   38|       Aronian|
+-----+--------------+



In [ ]:
pq.createOrReplaceTempView('q')
paw=spark.sql("select player_name from q order by count desc limit 1")

## **RESULT.14**

In [ ]:
paw.show()

+-----------+
|player_name|
+-----------+
|    Carlsen|
+-----------+



In [ ]:
df_14=paw.toPandas()
df_14

,player_name
0,Carlsen


15. List games where player has won game without queen.

Result attributes: game_id, event, player_name

In [ ]:
df_new5.createOrReplaceTempView('queen')

In [ ]:
df_new5.show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
p12=spark.sql("select df_5.game_id as game_id,df3.event,df3.first_name, df_5.white_queen_count,df_5.black_queen_count from df_5 INNER JOIN df3 ON df_5.game_id=df3.game_id;")

In [ ]:
p12.createOrReplaceTempView('queen')

In [ ]:
p12.show()

+--------------------+--------+----------+-----------------+-----------------+
|             game_id|   event|first_name|white_queen_count|black_queen_count|
+--------------------+--------+----------+-----------------+-----------------+
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|                1|                1|
|86e0b7f5-7b94-4ae...|WCh 2021|      draw|          

In [ ]:
q2=spark.sql("select * from queen where black_queen_count not like 1 and white_queen_count not like 1 ")

In [ ]:
q2.createOrReplaceTempView('queen')

In [ ]:
queen=spark.sql("select game_id,event,first_name as player from queen where first_name not like 'draw'")

## **Result.15**

In [ ]:
queen.show()

+--------------------+--------+-------+
|             game_id|   event| player|
+--------------------+--------+-------+
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
|55a122db-27d1-495...|WCh 2021|Carlsen|
+--------------------+--------+-------+
only showing top 20 rows



In [ ]:
df_15=queen.toPandas()
df_15

,game_id,event,player
0,55a122db-27d1-495c-bec4-bc811c1e6950,WCh 2021,Carlsen
1,55a122db-27d1-495c-bec4-bc811c1e6950,WCh 2021,Carlsen
2,55a122db-27d1-495c-bec4-bc811c1e6950,WCh 2021,Carlsen
3,55a122db-27d1-495c-bec4-bc811c1e6950,WCh 2021,Carlsen
4,55a122db-27d1-495c-bec4-bc811c1e6950,WCh 2021,Carlsen
...,...,...,...
1877,98231cc8-e2f8-4719-ae42-cfd6666ceb4d,WCh-FIDE,Topalov
1878,98231cc8-e2f8-4719-ae42-cfd6666ceb4d,WCh-FIDE,Topalov
1879,98231cc8-e2f8-4719-ae42-cfd6666ceb4d,WCh-FIDE,Topalov
1880,98231cc8-e2f8-4719-ae42-cfd6666ceb4d,WCh-FIDE,Topalov
